# Network with Applications

In this file, we'll demonstrate the simulation of a more complicated network topology with randomized applications. These applications will act on each node, first choosing a random other node from the network and then requesting a random number of entangled pairs between the local and distant nodes. The network topology, including hardware components, is shown below:

<img src="./notebook_images/star_network.png" width="700"/>

## Example

In this example, we construct the network described above and add the random request app included in SeQUeNCe. We'll be building the topology from an external json file `star_network.json`.

### Imports
We must first import the necessary tools from SeQUeNCe.
- `Timeline` is the main simulation tool, providing an interface for the discrete-event simulation kernel.
- `Topology` is a powerful class for creating and managing complex network topologies. We'll be using the `RouterNetTopo` subclass to build our network and intefrace with specific nodes and node types.
- `RandomRequestApp` is an example application included with SeQUeNCe. We will investigate its behavior when we add applications to our network.

In [1]:
import pandas as pd
from ipywidgets import interact
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.router_net_topo import RouterNetTopo
from sequence.app.random_request import RandomRequestApp

### Building the Simulation

We'll now construct the network and add our applications. This example follows the usual process to ensure that all tools function properly:
1. Create the simulated network topology. In this case, we are using an external JSON file to specify nodes and their connectivity.
    - This includes specifying hardware and timeline parameters in the custom `set_parameters` function, defined later
2. Install custom protocols/applications and ensure all are set up properly
3. Initialize and run the simulation
4. Collect and display the desired metrics

The JSON file specifies that network nodes should be of type `QuantumRouter`, a node type defined by SeQUeNCe. This will automatically create all necessary hardware and protocol instances on the nodes, and the `Topology` class will automatically generate `BSMNode` instances on the quantum channels between such nodes.

To construct an application, we need:
- The node to attach the application to
- The names (given as strings) of other possible nodes to generate links with
- A seed for the internal random number generator of the application

And, specifically for the random request app, we need:
- the minimum and maximum duration of the reservation
- the minimum and maximum amount of memories to reserve
- the minimum and maximum allowable fidelity for memories.

We can get a list of all desired application nodes, in this case routers, from the `Topology` class with the `get_nodes_by_type` method. We then set an application on each one, with the other possible connections being every other node in the network. We also give a unique random seed `i` to each application.

In [3]:
def test(sim_time, qc_atten):
    """
    sim_time: duration of simulation time (ms)
    qc_atten: quantum channel attenuation (dB/km)
    """
    network_config = "star_network.json"

    # here, we make a new topology using the configuration JSON file.
    # we then modify some of the simulation parameters of the network.
    network_topo = RouterNetTopo(network_config)
    set_parameters(network_topo, sim_time, qc_atten)
    
    # construct random request applications
    quantum_router_nodes = network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER)
    node_names = [node.name for node in quantum_router_nodes]
    apps = []
    for i, (name, node) in enumerate(zip(node_names, quantum_router_nodes)):
        # copy node name list
        other_nodes = node_names[:]
        other_nodes.remove(name)
        
        # create our application
        # arguments are:
        #   the host node,
        #   possible destination node names,
        #   a seed for the random number generator, and
        #   several arguments for request parameters.
        app = RandomRequestApp(node, other_nodes, i,
                               min_dur=1e13, max_dur=2e13, min_size=10,
                               max_size=25, min_fidelity=0.8, max_fidelity=1.0)
        apps.append(app)
        app.start()
    
    # run the simulation
    tl = network_topo.get_timeline()
    tl.show_progress = True
    tl.init()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    for app in apps:
        print("node " + app.node.name)
        print("\tnumber of wait times: ", len(app.get_wait_time()))
        print("\twait times:", app.get_wait_time())
        print("\treservations: ", app.reserves)
        print("\tthroughput: ", app.get_throughput())
    
    # create a table to showcase information about the reservations
    print("\nReservations Table:\n")
    node_names = []
    start_times = []
    end_times = []
    memory_sizes = []
    for node in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node_name = node.name
        for reservation in node.network_manager.protocol_stack[1].accepted_reservation:
            s_t, e_t, size = reservation.start_time, reservation.end_time, reservation.memory_size
            if reservation.initiator != node.name and reservation.responder != node.name:
                size *= 2
            node_names.append(node_name)
            start_times.append(s_t)
            end_times.append(e_t)
            memory_sizes.append(size)
    log = {"Node": node_names, "Start_time": start_times, "End_time": end_times, "Memory_size": memory_sizes}
    df = pd.DataFrame(log)
    print(df)

### Setting parameters

Here we define the `set_parameters` function we used earlier. This function will take a `Topology` as input and change many parameters to desired values.

The simulation time limit will be set using the `get_timeline` method.

Quantum memories and detectors are hardware elements, and so parameters are changed by accessing the hardware included with the `QuantumRouter` and `BSMNode` node types. Many complex hardware elements, such as bsm devices or memory arrays, have methods to update parameters for all included hardware elements. This includes `update_memory_params` to change all memories in an array or `update_detector_params` to change all detectors.

We will also set the success probability and swapping degradation of the entanglement swapping protocol. This will be set in the Network management Module (specifically the reservation protocol), as this information is necessary to create and manage the rules for the Resource Management module.

Lastly, we'll update some parameters of the quantum channels. Quantum channels (and, similarly, classical channels) can be accessed from the `Topology` class as the `qchannels` field. Since these are individual hardware elements, we will set the parameters directly.

In [4]:
def set_parameters(topology, simulation_time, attenuation):
    """
    simulation_time: duration of simulation time (in ms)
    attenuation: attenuation on quantum channels (in db/m)
    """
    
    PS_PER_MS = 1e9
    
    # set timeline stop time
    topology.get_timeline().stop_time = (simulation_time * PS_PER_MS)
    
    # set memory parameters
    MEMO_FREQ = 2e3
    MEMO_EXPIRE = 0
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9349367588934053
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        memory_array = node.get_components_by_type("MemoryArray")[0]
        memory_array.update_memory_params("frequency", MEMO_FREQ)
        memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set detector parameters
    DETECTOR_EFFICIENCY = 0.9
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type(RouterNetTopo.BSM_NODE):
        bsm = node.get_components_by_type("SingleAtomBSM")[0]
        bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)
        
    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.90
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)
        
    # set quantum channel parameters
    ATTENUATION = attenuation
    QC_FREQ = 1e11
    for qc in topology.qchannels:
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

### Running the Simulation

All that is left is to run the simulation with user input. We'll specify:

    sim_time: duration of simulation time (ms)
    qc_atten: attenuation on quantum channels (db/m)

Note that different hardware parameters or network topologies may cause the simulation to run for a very long time.

In [5]:
interact(test, sim_time=50e3, qc_atten=[0, 1e-5, 2e-5])

interactive(children=(FloatSlider(value=50000.0, description='sim_time', max=150000.0, min=-50000.0), Dropdown…

<function __main__.test(sim_time, qc_atten)>